In [1]:

import pandas as pd
from predict_protein import download_cptac, select_features, train_model
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, ElasticNetCV

In [2]:
cptac_list = download_cptac.download_cptac(n_tumors=7)

cptac warning: transcriptomics data was not found for the following samples, so transcriptomics data columns were filled with NaN for these samples: C3L-00563.N, C3L-00605.N, C3L-00769.N, C3L-00770.N, C3L-00771.N, C3L-00930.N, C3L-00947.N, C3L-00963.N, C3L-01246.N, C3L-01249.N, C3L-01252.N, C3L-01256.N, C3L-01257.N, C3L-01744.N, C3N-00200.N, C3N-00729.N, C3N-01211.N, NX1.N, NX10.N, NX11.N, NX12.N, NX13.N, NX14.N, NX15.N, NX16.N, NX17.N, NX18.N, NX2.N, NX3.N, NX4.N, NX5.N, NX6.N, NX7.N, NX8.N, NX9.N (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac.py, line 38)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: 01OV002, 01OV008, 01OV010, 01OV013, 01OV019, 01OV045, 01OV046, 01OV049, 02OV035, 02OV040, 02OV042, 02OV045, 04OV041, 11OV009, 13OV004, 17OV019, 17OV034, 22OV001, 26OV010 (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac.py, line 42)
cptac

In [3]:
tm = train_model.LearnCPTAC(pd.concat(cptac_list))

In [4]:
tm.train_method = 'linreg'
tm.included_features = 'string'

In [5]:
# Problematic proteins: tumor 7 linreg string 7027 RABL6 ...
tm.shared_proteins[7027]

'RABL6'

In [7]:
# This worked in string earlier
res = tm.learn_one_protein('RABL6')
res

{'model': LinearRegression(n_jobs=-1),
 'metrics':        corr_train  corr_test  r2_train   r2_test  num_obs  num_features  \
 RABL6    0.853652   0.462066  0.728722 -0.061254      864           334   
 
           nrmse  baseline_nrmse  
 RABL6  0.241967        0.236143  }

In [ ]:
tm.get_train_test('RFXANK')

In [ ]:
string_interactors = tm.stringdb.find_interactor('RFXANK',
                                                   combined_score=200,
                                                   max_proteins=1000,
                                                   )
proteins_to_include = [p for p in string_interactors if p in tm.shared_proteins]
proteins_to_include

In [ ]:
protein_to_do = 'RFXANK'
y_df = tm.df[[protein_to_do + '_proteomics']]
y_df = y_df.dropna(subset=[protein_to_do + '_proteomics'])
len(y_df)
y_df

In [ ]:
# proteins_to_include = [protein_to_do]
x_df = tm.df[[tx + '_transcriptomics' for tx in proteins_to_include]]
x_df

In [ ]:
x_df = x_df.dropna(axis=1, how='all').copy()

In [ ]:
x_df.shape[1]

In [ ]:
x_impute = SimpleImputer(missing_values=np.nan, strategy='median').fit_transform(x_df)

In [ ]:
x_df.columns

In [ ]:
x_impute_df = pd.DataFrame(x_impute, columns=x_df.columns, index=x_df.index)

In [ ]:
# remove low variance columns
try:
    vt = VarianceThreshold().fit(x_impute_df)

# catch instances where no column has any variance
except ValueError:
    pass

x_impute_var_df = x_impute_df.iloc[:, np.where(vt.variances_ > tm.var_threshold)[0]]

In [ ]:
x_impute_var_df